In [1]:
import spacy
import pickle
import random
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from spacy.training.example import Example
!pip install PyMuPDF
import sys, fitz

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB 262.6 kB/s eta 0:00:13
   - -------------------------------------- 0.1/3.2 MB 508.4 kB/s eta 0:00:07
   --- ------------------------------------ 0.2/3.2 MB 1.3 MB/s eta 0:00:03
   ------ --------------------------------- 0.5/3.2 MB 2.1 MB/s eta 0:00:02
   ------------ --------------------------- 1.0/3.2 MB 3.6 MB/s eta 0:00:01
   ----------------------- ---------------- 1.9/3.2 MB 6.1 MB/s eta 0:00:01
   --------------------------------- ------ 2.7/3.2 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/24.9 MB ? eta -:--:--
   -- ------------------------------------- 1.4/24.9 MB 42.2 MB/s eta 0:00:01
   -- ---------------------------

In [2]:
print(spacy.__version__)

3.7.4


In [6]:
train_data = pickle.load(open('train_data.pkl', 'rb'))

In [7]:
nlp = spacy.blank('en')

In [8]:
def train_model(nlp, train_data):
    # Check if the NER component is already in the pipeline
    if 'ner' not in nlp.pipe_names:
        ner = nlp.add_pipe('ner', last=True)
    else:
        ner = nlp.get_pipe('ner')

    # Add labels to the NER component
    for _, annotations in train_data:
        for ent in annotations['entities']:
            ner.add_label(ent[2])

    # Disable other pipelines
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()

        for epoch in range(10):
            print(f"Iteration {epoch} starting...")
            random.shuffle(train_data)
            losses = {}

            for text, annotations in train_data:
                try:
                    example = Example.from_dict(nlp.make_doc(text), annotations)
                    nlp.update([example], drop=0.5, losses=losses, sgd=optimizer)
                except Exception as e:
                    pass

            print(losses)

# Train the model
train_model(nlp, train_data)

Iteration 0 starting...
{'ner': 10696.012430361096}
Iteration 1 starting...
{'ner': 3303.5693580655516}
Iteration 2 starting...
{'ner': 2747.150214919343}
Iteration 3 starting...
{'ner': 2740.640053330895}
Iteration 4 starting...
{'ner': 2947.5751557315716}
Iteration 5 starting...
{'ner': 3677.713366558738}
Iteration 6 starting...
{'ner': 2554.4365836934903}
Iteration 7 starting...
{'ner': 2781.7435772676213}
Iteration 8 starting...
{'ner': 2202.5650490493417}
Iteration 9 starting...
{'ner': 2090.5911758059888}


In [9]:
nlp.to_disk('nlp_model2')

In [16]:
nlp_model = spacy.load('nlp_model2')

In [19]:
train_data[0][0]

'Puneeth R Escalation Specialist - HiPower Support Centre  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Puneeth-R/bc332220e733906d  To be an asset to my organization by utilizing the recent improving trends and make optimum use of the available resources for the success of the company. More importantly, working harmoniously in a team to bring forth the best from self and everyone.  Willing to relocate: Anywhere  WORK EXPERIENCE  Escalation Specialist  HiPower Support Centre -  June 2015 to Present  Client: Machine Zone (Supporting 3 Games) Roles &amp; Responsibilities: • Escalating tickets to clients, Creating Inquiries regarding the process to receive update from Clients, Providing feedback to Agents regarding Clients Escalations and internal invalid Escalations. Team handling ( Mentor) • Tracking performances of the team members in terms of Production, Quality and CSAT. • Monitoring Queue, Allocating Agents to different queues, Preparing Reports, Process improvements initi

In [17]:
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Puneeth R
DESIGNATION                   - Escalation Specialist
LOCATION                      - Bengaluru
DESIGNATION                   - Escalation Specialist
COMPANIES WORKED AT           - Cisco
COMPANIES WORKED AT           - Cisco
DEGREE                        - B.E
COLLEGE NAME                  - PES College of Engineering


In [27]:
fname = 'sample2.pdf'
doc = fitz.open(fname)
text = ""
for page in doc:
    text += page.get_text("text")
#replace new line with space
tx = " ".join(text.split('\n'))
print(tx)

Tansu Gangopadhyay https:/ /tansu.github.io tansu@gmail.com | +91 9836968608 EDUCATION SRM UNIVERSITY B.Tech in Computer Science July 2025| Chennai,India CGPA-8.52 KALYANI PUBLIC SCHOOL Graduation in Science August 2021 | Kolkata, India Percentage-82.1 LINKS Github:// tansugangopadhyay LinkedIn:// tansugangopadhyay Twitter:// @tansuganguly Website:// tansugangopadhyay LeetCode:// tansugangopadhyay HackerRank:// tansugangopadhyay Instagram:// tansugangopadhyay COURSEWORK UNIVERSITY Data Structure Object Oriented Programming Analogue and Digital Electronics Programming For Problem Solving Computer Organization Computer Architecture C++ SCHOOL C Python JavaScript SKILLS PROGRAMMING Over 2000 lines: Python • Solidity • C Over 1000 lines: C++ • Java • CSS • HTML • JavaScript Familiar: Assembly • Rust • MySQL VOLUNTEER EXPERIENCE TEAM ENVISION | Blockchain Developer Jan 2022 - Present | Chennai, India • Created a NFT generator which can generate upto 600 unique NFT’s in less than 60 seconds 

In [28]:
doc = nlp_model(tx)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

NAME                          - Tansu Gangopadhyay https:/ /tansu.github.io tansu@gmail.com | +91 9836968608 EDUCATION SRM UNIVERSITY
DEGREE                        - B.Tech in Computer Science July 2025| Chennai,India CGPA-8.52 KALYANI PUBLIC SCHOOL Graduation in Science August 2021
LOCATION                      - Kolkata
